In [ ]:
import numpy as np
import tensorflow as tf
import pandas as pd
from tensorflow.keras.layers import Dense, LeakyReLU, Input, BatchNormalization, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# reading values
data = pd.read_csv('data.csv').values

def normalize(data):
    min_data = np.min(data, axis=0)
    max_data = np.max(data, axis=0)
    return 2 * (data - min_data) / (max_data - min_data) - 1

def rescale(data, original_data):
    min_data = np.min(original_data, axis=0)
    max_data = np.max(original_data, axis=0)
    return 0.5 * (data + 1) * (max_data - min_data) + min_data

# defining generator
def create_generator():
    input_layer = Input(shape=(100,))
    x = Dense(32)(input_layer)
    x = LeakyReLU(alpha=0.2)(x)
    x = BatchNormalization()(x)
    x = Dense(64)(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = BatchNormalization()(x)
    x = Dense(128)(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = BatchNormalization()(x)
    output_layer = Dense(5, activation='tanh')(x)
    model = Model(inputs=input_layer, outputs=output_layer)
    return model

# defining discriminator
def create_discriminator():
    input_layer = Input(shape=(5,))
    x = Dense(32)(input_layer)
    x = LeakyReLU(alpha=0.2)(x)
    x = Dropout(0.3)(x)
    x = Dense(64)(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Dropout(0.3)(x)
    x = Dense(128)(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Dropout(0.3)(x)
    output_layer = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer=Adam(learning_rate=0.0002, beta_1=0.5), loss='binary_crossentropy', metrics=['accuracy'])
    return model

#defining 'GAN'
def create_gan(generator, discriminator):
    discriminator.trainable = False
    gan_input = Input(shape=(100,))
    gan_output = discriminator(generator(gan_input))
    model = Model(inputs=gan_input, outputs=gan_output)
    model.compile(optimizer=Adam(learning_rate=0.0002, beta_1=0.5), loss='binary_crossentropy')
    return model

# create the generator variable
generator = create_generator()
# create the discriminator variable
discriminator = create_discriminator()
# create gan variable
gan = create_gan(generator, discriminator)

def train_gan(epochs, batch_size):
    for e in range(epochs):
        for _ in range(batch_size):
            # generate random noise
            noise = np.random.normal(0, 1, (batch_size, 100))

            idx = np.random.randint(0, data.shape[0], size=batch_size)
            real_data = data[idx]

            synthetic_data = generator.predict(noise)

            combined_data = np.concatenate((real_data, synthetic_data))
            labels = np.concatenate((np.ones(batch_size), np.zeros(batch_size)))

            # Shuffle the data before training
            p = np.random.permutation(len(combined_data))
            combined_data = combined_data[p]
            labels = labels[p]
            d_loss = discriminator.train_on_batch(combined_data, labels)
            gan_input = np.random.normal(0, 1, (batch_size, 100))
            gan_labels = np.ones(batch_size)
            g_loss = gan.train_on_batch(gan_input, gan_labels)

        print(f"Epoch {e+1}/{epochs}, Discriminator Loss: {d_loss[0]}, Generator Loss: {g_loss}")
        


# Normalize the original data
normalized_data = normalize(data)
        
        
train_gan(epochs=5000, batch_size=32)

synthetic_datasets = []

for _ in range(500):
    random_input = np.random.normal(0, 1, (1, 100))
    synthetic_data = generator.predict(random_input)
    synthetic_datasets.append(synthetic_data)

synthetic_datasets = np.vstack(synthetic_datasets)

rescaled_synthetic_datasets = rescale(synthetic_datasets, data)

pd.DataFrame(synthetic_datasets, columns=
             ['Cellulose (w/w)', 'Hemicellulose (w/w)', 'Lignin (w/w)', 'Residual (w/w)', 'Biomethane Potential (I CH4/Kg VS)']).to_csv('synthetic-data2.csv', index=False)


1/1 [==============================] - 0s 14ms/step
Epoch 1/5000, Discriminator Loss: 4.700963973999023, Generator Loss: 0.691963255405426
1/1 [==============================] - 0s 15ms/step
Epoch 2/5000, Discriminator Loss: 0.6679320335388184, Generator Loss: 0.688685417175293
1/1 [==============================] - 0s 17ms/step
Epoch 3/5000, Discriminator Loss: 1.0137823820114136, Generator Loss: 0.6930637359619141
1/1 [==============================] - 0s 13ms/step
Epoch 4/5000, Discriminator Loss: 0.31925588846206665, Generator Loss: 0.7317637801170349
1/1 [==============================] - 0s 14ms/step


1/1 [==============================] - 0s 25ms/step
Epoch 5/5000, Discriminator Loss: 0.47514185309410095, Generator Loss: 0.8033459782600403
1/1 [==============================] - 0s 16ms/step
Epoch 6/5000, Discriminator Loss: 0.40019771456718445, Generator Loss: 0.921927809715271
1/1 [==============================] - 0s 16ms/step
Epoch 7/5000, Discriminator Loss: 0.2480689138174057, Generator Loss: 0.8670289516448975
1/1 [==============================] - 0s 14ms/step
Epoch 8/5000, Discriminator Loss: 0.25683754682540894, Generator Loss: 0.9065312147140503
1/1 [==============================] - 0s 19ms/step
Epoch 9/5000, Discriminator Loss: 0.24877554178237915, Generator Loss: 1.003932237625122
1/1 [==============================] - 0s 14ms/step


1/1 [==============================] - 0s 17ms/step
Epoch 10/5000, Discriminator Loss: 0.2252991646528244, Generator Loss: 0.9790570735931396
1/1 [==============================] - 0s 13ms/step
Epoch 11/5000, Discriminator Loss: 0.21895365417003632, Generator Loss: 1.0846582651138306
1/1 [==============================] - 0s 13ms/step
Epoch 12/5000, Discriminator Loss: 0.20700271427631378, Generator Loss: 1.126627802848816
1/1 [==============================] - 0s 14ms/step
Epoch 13/5000, Discriminator Loss: 0.18423324823379517, Generator Loss: 1.1754014492034912
1/1 [==============================] - 0s 14ms/step
Epoch 14/5000, Discriminator Loss: 0.1711580902338028, Generator Loss: 1.2930126190185547


1/1 [==============================] - 0s 13ms/step
Epoch 15/5000, Discriminator Loss: 0.15998320281505585, Generator Loss: 1.3963414430618286
1/1 [==============================] - 0s 14ms/step
Epoch 16/5000, Discriminator Loss: 0.14134113490581512, Generator Loss: 1.3344568014144897
1/1 [==============================] - 0s 70ms/step
Epoch 17/5000, Discriminator Loss: 0.27195531129837036, Generator Loss: 1.4646399021148682
1/1 [==============================] - 0s 15ms/step
Epoch 18/5000, Discriminator Loss: 0.13663627207279205, Generator Loss: 1.538825273513794
1/1 [==============================] - 0s 14ms/step


1/1 [==============================] - 0s 15ms/step
Epoch 19/5000, Discriminator Loss: 0.11967569589614868, Generator Loss: 1.6228744983673096
1/1 [==============================] - 0s 14ms/step
Epoch 20/5000, Discriminator Loss: 0.106023408472538, Generator Loss: 1.657529354095459
1/1 [==============================] - 0s 15ms/step
Epoch 21/5000, Discriminator Loss: 0.10307285189628601, Generator Loss: 1.811537265777588
1/1 [==============================] - 0s 14ms/step
Epoch 22/5000, Discriminator Loss: 0.09989321231842041, Generator Loss: 1.8115111589431763
1/1 [==============================] - 0s 17ms/step
Epoch 23/5000, Discriminator Loss: 0.09130696952342987, Generator Loss: 1.8698506355285645
1/1 [==============================] - 0s 74ms/step


...

1/1 [==============================] - 0s 19ms/step
Epoch 477/5000, Discriminator Loss: 8.18310581962578e-07, Generator Loss: 19.841087341308594
1/1 [==============================] - 0s 16ms/step
Epoch 478/5000, Discriminator Loss: 1.1105494195362553e-06, Generator Loss: 20.80971908569336
1/1 [==============================] - 0s 16ms/step
Epoch 479/5000, Discriminator Loss: 3.8993368889350677e-07, Generator Loss: 19.275381088256836
1/1 [==============================] - 0s 16ms/step
Epoch 480/5000, Discriminator Loss: 1.258281736227218e-05, Generator Loss: 19.661544799804688
1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 0s/step
Epoch 481/5000, Discriminator Loss: 1.1048858823414776e-06, Generator Loss: 20.16014862060547
1/1 [==============================] - 0s 7ms/step
Epoch 482/5000, Discriminator Loss: 2.5336561293443083e-07, Generator Loss: 20.349966049194336
1/1 [==============================] - 0s 16ms/step
Epoch 483/5000, Discriminator Loss: 7.920561984064989e-07, Generator Loss: 20.45566177368164
1/1 [==============================] - 0s 16ms/step
Epoch 484/5000, Discriminator Loss: 2.9933357836853247e-07, Generator Loss: 20.04573631286621
1/1 [==============================] - 0s 16ms/step
Epoch 485/5000, Discriminator Loss: 7.319872565858532e-06, Generator Loss: 20.385364532470703
1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 29ms/step
Epoch 486/5000, Discriminator Loss: 1.0132532679563155e-07, Generator Loss: 19.790496826171875
1/1 [==============================] - 0s 16ms/step
Epoch 487/5000, Discriminator Loss: 2.169174194932566e-06, Generator Loss: 20.10921859741211
1/1 [==============================] - 0s 16ms/step
Epoch 488/5000, Discriminator Loss: 1.0247894977055694e-07, Generator Loss: 20.160980224609375
1/1 [==============================] - 0s 16ms/step
Epoch 489/5000, Discriminator Loss: 7.18447239478337e-08, Generator Loss: 20.46253204345703
1/1 [==============================] - 0s 16ms/step
Epoch 490/5000, Discriminator Loss: 9.193218488690036e-07, Generator Loss: 19.951770782470703


1/1 [==============================] - 0s 16ms/step
Epoch 491/5000, Discriminator Loss: 9.82046231001732e-08, Generator Loss: 20.079002380371094
1/1 [==============================] - 0s 16ms/step
Epoch 492/5000, Discriminator Loss: 2.1158842855584226e-07, Generator Loss: 19.997589111328125
1/1 [==============================] - 0s 14ms/step
Epoch 493/5000, Discriminator Loss: 7.267590262927115e-08, Generator Loss: 19.810726165771484
1/1 [==============================] - 0s 0s/step
Epoch 494/5000, Discriminator Loss: 6.624478601224837e-07, Generator Loss: 20.258743286132812
1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 16ms/step
Epoch 495/5000, Discriminator Loss: 5.604217108157172e-07, Generator Loss: 19.452421188354492
1/1 [==============================] - 0s 16ms/step
Epoch 496/5000, Discriminator Loss: 2.1217049095412222e-07, Generator Loss: 20.270431518554688
1/1 [==============================] - 0s 16ms/step
